In [9]:
import pandas as pd
import seaborn as sns
## loads in newly created csv training files
driving = pd.read_csv("training csv/garminlongdrive.csv")
walking=pd.read_csv("training csv/sergiolongwalk.csv")
driving2=pd.read_csv("training csv/sergioparkinglot.csv")
biking= pd.read_csv("training csv/bessbike.csv")


In [10]:
import statistics as stats
## adds variance of previous 2, 5 and 10 data points, used for classification
def addvariance(tracking_df):
    vartwo=[]
    varfive=[]
    varten=[]
    for i,j in enumerate(tracking_df.mph):
        var=0
        if i>=2:
            previoustwo = [j,tracking_df.mph[i-1]]
            var=stats.variance(previoustwo)
        vartwo.append(var)
        if i>=5:
            previousfive = [j,tracking_df.mph[i-1],tracking_df.mph[i-2],tracking_df.mph[i-3],tracking_df.mph[i-4]]
            var=stats.variance(previousfive)
        varfive.append(var)
        if i>=10:
            previousten = [j,tracking_df.mph[i-1],tracking_df.mph[i-2],tracking_df.mph[i-3],tracking_df.mph[i-4],tracking_df.mph[i-5],tracking_df.mph[i-6],tracking_df.mph[i-7],tracking_df.mph[i-9]]
            var=stats.variance(previousten)
        
        varten.append(var)
    tracking_df["variance2"]=vartwo
    tracking_df["variance5"]=varfive
    tracking_df["variance10"]=varten
    
addvariance(walking)
addvariance(driving)
addvariance(driving2)
addvariance(biking)
driving["Class"]=2
driving2["Class"]=2
walking["Class"]=0
biking["Class"]=1
tracking_df=walking.append(driving[0:500]).append(driving2).append(biking).append(biking).append(biking)
tracking_df=tracking_df.reset_index(drop=True)
tracking_df=tracking_df.drop(["latitude","longitude","time","distanceinmeters","HHMMSS","meters","seconds","heartrate"],axis=1)

In [11]:
import matplotlib.pyplot as plt  
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import normalize
from sklearn.neighbors import KNeighborsClassifier
## creates classification algorithm (KNN)
def modelrunner(df):
    randomized = (df.sample(frac=1).reset_index(drop=True)).to_numpy()
    X=randomized[:,:-1]
    X=normalize(X, norm='max', axis=0)
    y=randomized[:,-1:][:,0] 
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=1000/(len(y)), random_state=42)

    knn =  KNeighborsClassifier(n_neighbors=5)

    knn.fit(X_train,Y_train)
    
    Y_pred=knn.predict(X_test)
    print("training accuracy score is ", metrics.accuracy_score(Y_test,Y_pred))

    return knn

In [12]:
knn = modelrunner(tracking_df)

training accuracy score is  0.9533582089552238


In [13]:
## function to be used on the the actual data, classifies transportation type using variance of speeds and current speed
def classifier(tracking_df):
    df=tracking_df.drop(["latitude","longitude","time","distanceinmeters","HHMMSS","meters","seconds","heartrate"],axis=1)
    test=normalize(df.to_numpy(), norm='max', axis=0)
    y=knn.predict(test)
    print(len(y))
    tracking_df["Transport"]=y
    tracking_df=tracking_df.drop(["distanceinmeters","heartrate","HHMMSS","meters","seconds","variance2","variance5","variance10"],axis=1)
    return tracking_df


# Upload tracking tcx files here to classify transportation type and add speed

In [14]:
import ipywidgets as widgets
import pandas as pd
import codecs
import xml.etree.ElementTree as ET 

In [15]:

upload=widgets.FileUpload(
    accept='.tcx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
upload

FileUpload(value={}, accept='.tcx', description='Upload')

In [16]:
name=list(upload.value)[0]
byte=upload.value[name]["content"]
txt=codecs.decode(byte, encoding="utf-8")
xroot = ET.fromstring(txt)

## convert tcx to df
df_cols = ["time","latitude","longitude","distanceinmeters","heartrate"]
rows = []
ctr=0
for lap in xroot[0][0].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Lap"):
    for node in lap[8].findall("{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint"):
        if node[1].tag == "{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position":
            time = node[0].text
            lat = node[1][0].text
            long = node[1][1].text
            dist= node[3].text
            bpm = node[4][0].text

            rows.append({"time": time, "latitude": lat, 
                            "longitude": long, "distanceinmeters": dist,
                            "heartrate":bpm})

    tracking_df = pd.DataFrame(rows, columns = df_cols)
## format time  
def time_splitter (row):
   datetime=row['time'].split("T")
   return datetime[1]
  
tracking_df["HHMMSS"]=tracking_df.apply (lambda row: time_splitter(row), axis=1)

tracking_df.distanceinmeters=tracking_df.distanceinmeters.astype(str).astype(float)

## add meters column
meterscol=[]
for i,j in enumerate(tracking_df.distanceinmeters):
    meters=0
    if i>0:
        lastmet=tracking_df.distanceinmeters[i-1]
        met =j
        meters=met-lastmet
    meterscol.append(meters)


tracking_df["meters"]=meterscol

##add seconds column
secondscol=[]
for i,j in enumerate(tracking_df.HHMMSS):
    seconds=0
    if i>0:
        lastmins=int(tracking_df.HHMMSS[i-1][3:5])
        lastsecs=int(tracking_df.HHMMSS[i-1][6:8])
        mins=int(j[3:5])
        secs=int(j[6:8])
        if lastmins==mins:
            seconds=secs-lastsecs
        else:
            seconds = (60-lastsecs)+secs
    secondscol.append(seconds)
    
tracking_df["seconds"]=secondscol

## calculates mph with meters and seconds
def calculate_speed(row):
    if (row.seconds>0):
        meterspersecond=row.meters/row.seconds
        milesperhour=meterspersecond*2.237
        return milesperhour
    else: return 0

## classifies with knn algorithm
tracking_df["mph"]=tracking_df.apply (lambda row: calculate_speed(row), axis=1)
addvariance(tracking_df)
tracking_df = tracking_df[tracking_df.mph < 85]
testingclassified=classifier(tracking_df)

2235


In [17]:
filename=name.split(".")[0]

In [18]:
##places finalized fully classified csv in trials folder to be input into arcgis
testingclassified.to_csv(r"trials/"+filename+".csv", index = False)